In [1]:
import biovec
import numpy as np

from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score

In [2]:
# TASK 1 - Generating corpus file from fasta file and train word2vec
'''model = biovec.ProtVec(corpus_fname="uniprot_sprot.fasta", 
                    n=3,
                    size=100,
                    window=5,
                    sg=1,
                    min_count=2)
    
model.save('./saved_model')'''

'model = biovec.ProtVec(corpus_fname="uniprot_sprot.fasta", \n                    n=3,\n                    size=100,\n                    window=5,\n                    sg=1,\n                    min_count=2)\n    \nmodel.save(\'./saved_model\')'

In [3]:
# Load saved model
pv = biovec.models.load_protvec('./saved_model')

#pv = biovec.models.load_protvec("swissprot-reviewed.model")

In [4]:
from scipy import spatial
from numpy import dot
from numpy.linalg import norm

AAA = "-0.40824547 -0.071329094 -0.064001456 -0.05630998 -0.2195283 0.009979812 0.17874652 0.08375562 0.027502276 -0.07130323 0.006456163 0.0030929213 0.08790806 0.0032573056 -0.0698815 0.0905431 0.15583898 0.110263005 -0.22512965 -0.001306174 -0.109739184 0.063375674 0.32993525 0.13893794 -0.036341757 0.03819593 0.14535004 -0.0073543214 0.13369273 -0.1429071 -0.16775587 -0.041495405 0.13457869 -0.12228688 -0.10785071 0.0071128756 0.22321464 -0.058188416 0.20746201 0.03532325 -0.019520825 -0.059025783 0.295711 0.1075264 0.13401115 -0.3459603 0.035944186 0.13798769 -0.115936905 -0.032773007 -0.13372639 0.14214544 -0.30464968 -0.10083931 -0.0861974 -0.38257608 -0.0012935519 -0.01750991 0.01609032 0.16624023 -0.11012829 0.18223743 -0.04707044 0.07104323 -0.23681891 -0.3489856 -0.0397025 0.15314193 -0.05234596 0.087228335 0.047370806 0.025876798 0.25509173 -0.042357408 -0.37494487 0.08057458 0.042824224 -0.19328211 0.19914825 0.009554409 0.16090411 0.16013925 0.14780127 -0.39027962 0.41453123 0.030759456 -0.25644022 0.30203447 -0.1973766 0.19089536 -0.14282611 -0.36060184 -0.03375458 0.008354174 0.11964532 -0.18391272 0.35435116 -0.1552305 0.13506724 -0.13348278"
tmp = AAA.split(" ")
a = [float(i) for i in tmp]
b = pv["LRX"]

result = 1 - spatial.distance.cosine(a, b)
print(result)

cos_sim = dot(a, b)/(norm(a)*norm(b))
print(cos_sim)

0.031409058550162516
0.03140905624381981


/home/francesco/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [5]:
# Prepare dataset for prediction
protein_names, sequences, labels = [], [], []


'''
    Labels:
        "+" stands for "binding protein" => 1
        "-" stands for "non-binding" => 0
'''
def convert_label(label_string):
 
    if label_string == "+":
        return 1
    elif label_string == "-":
        return 0
    else:
        return None

    
# Open file containing dataset    
with open('./ppi_data.fasta') as f:
    lines = f.read().splitlines()
    
    for i in range(len(lines)):
        
        if i % 3 == 0:
            protein_names.append(lines[i])
        elif i % 3 == 1:
            sequences.append(lines[i])
        elif i % 3 == 2:
            labels.append([convert_label(letter) for letter in lines[i]])
            
protein_names = np.array(protein_names)
sequences = np.array(sequences)
labels = np.array(labels)

assert(protein_names.shape[0] == sequences.shape[0] == labels.shape[0])

print(protein_names[0])
print(sequences[0])
print(labels[0])

>P0A8Q6
MGKTNDWLDFDQLAEEKVRDALKPPSMYKVILVNDDYTPMEFVIDVLQKFFSYDVERATQLMLAVHYQGKAICGVFTAEVAETKVAMVNKYARENEHPLLCTLEKA
[0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]


### Example for sequence   M(VWL|N|GEP)RPLEG ...
    - Window size 7
    - Center letter is N
    - Vec(VWLNGEP) = Vec(VWL) + Vec(WLN) + Vec(LNG) + Vec(NGE) + Vec(GEP)

In [6]:
window_size = 7
vector_size = 100

X, y = [], []

# Iterates over all proteins in dataset
for i in range(len(sequences)):

    # Loop over sequence
    for j in range(0,len(sequences[i]) - window_size + 1):
        sub_sequence = sequences[i][j:j+window_size]

        # Sum vectors of 3 grams
        tmp = np.zeros(vector_size)
        for k in range(0, window_size - 2):
            tmp = tmp + np.array(pv[sub_sequence[k:k+3]])
    
        X.append(tmp)
        y.append(labels[i][j+3])    
        
X, y = np.array(X), np.array(y)

print(X.shape)
print(y.shape)

/home/francesco/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  app.launch_new_instance()


(89541, 100)
(89541,)


In [7]:
# Cross validation in 10-Folds
skf = StratifiedKFold(n_splits=10, random_state=42, shuffle=True)
skf.get_n_splits(X, y)


hyperparameters = {
    'hidden_layer_sizes': [(25), (50), (100)],
    'learning_rate_init': [0.01, 0.001],
    'max_iter': [100, 200, 500]
}

accuracy, precision, recall, auc = [], [], [], []

for train_index, test_index in skf.split(X, y):
    print("TRAIN indeces:", train_index.shape, "TEST indeces:", test_index.shape)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    estimator = GridSearchCV(
        MLPClassifier(random_state=42),
        param_grid=hyperparameters, 
        n_jobs=-1,
        verbose=0)
    
    # Training
    estimator.fit(X_train, y_train)
    print(estimator.best_params_)
    
    # Predict on test data
    #predictions = estimator.predict(X_test)
    predictions = estimator.best_estimator_.predict(X_test)

    # Scores
    accuracy.append(accuracy_score(y_test, predictions))
    precision.append(precision_score(y_test, predictions))
    recall.append(recall_score(y_test, predictions, average='micro'))
    auc.append(roc_auc_score(y_test, predictions))
    
    print(accuracy)
    
    
print(np.mean(np.array(accuracy)))
print(np.mean(np.array(precision)))
print(np.mean(np.array(recall)))
print(np.mean(np.array(auc)))
    

TRAIN indeces: (80586,) TEST indeces: (8955,)


/home/francesco/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/francesco/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/francesco/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


{'hidden_layer_sizes': 25, 'learning_rate_init': 0.01, 'max_iter': 100}
[0.7206030150753768]
TRAIN indeces: (80586,) TEST indeces: (8955,)


/home/francesco/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/francesco/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/francesco/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/francesco/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn'

{'hidden_layer_sizes': 25, 'learning_rate_init': 0.01, 'max_iter': 100}
[0.7206030150753768, 0.7221663874930206]
TRAIN indeces: (80586,) TEST indeces: (8955,)


...

{'hidden_layer_sizes': 50, 'learning_rate_init': 0.01, 'max_iter': 100}
[0.7206030150753768, 0.7221663874930206, 0.7222780569514238, 0.721608040201005, 0.7204601295510387, 0.7216886307795398, 0.7225820862184499, 0.7195353512789009]
TRAIN indeces: (80588,) TEST indeces: (8953,)


/home/francesco/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/francesco/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/francesco/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


{'hidden_layer_sizes': 25, 'learning_rate_init': 0.01, 'max_iter': 100}
[0.7206030150753768, 0.7221663874930206, 0.7222780569514238, 0.721608040201005, 0.7204601295510387, 0.7216886307795398, 0.7225820862184499, 0.7195353512789009, 0.7195353512789009]
TRAIN indeces: (80588,) TEST indeces: (8953,)


/home/francesco/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/francesco/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/francesco/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


{'hidden_layer_sizes': 25, 'learning_rate_init': 0.01, 'max_iter': 100}
[0.7206030150753768, 0.7221663874930206, 0.7222780569514238, 0.721608040201005, 0.7204601295510387, 0.7216886307795398, 0.7225820862184499, 0.7195353512789009, 0.7195353512789009, 0.7173014631966939]
0.720775851202435
0.5111729424570868
0.720775851202435
0.510207754657122
